In [ ]:
import os
import tensorflow.compat.v1 as tf
import pandas as pd
import cv2
import numpy as np
from PIL import Image
from input_pipeline import Pipeline

# Define a graph

In [ ]:
tf.reset_default_graph()

dataset_path = '/home/dan/datasets/activity_net_subset/shards/'
filenames = os.listdir(dataset_path)
filenames = [n for n in filenames if n.endswith('.tfrecords')]
filenames = [os.path.join(dataset_path, n) for n in sorted(filenames)]

pipeline = Pipeline(filenames, is_training=True, batch_size=16)
dataset = pipeline.dataset
iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
init = iterator.make_initializer(dataset)
x = iterator.get_next()
print(x)

# Show a sample

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    y = sess.run(x)
    # it has shape [16, 4, 256, 256, 2]

In [ ]:
DECODER = np.load('decoder.npy')

def show_batch(y):
    image = Image.new('RGB', (256 * 4 * 2, 256 * 16))
    for i in range(16):

        gray = y[i, :, :, :, 0]
        gray = np.concatenate(gray, axis=1)
        # it has shape [256, 4 * 256]

        codes = y[i, :, :, :, 1]
        codes = np.concatenate(codes, axis=1)
        colors = DECODER[codes]
        # it has shape [256, 4 * 256, 2]

        gray_constant = 200 * np.ones_like(gray)
        gray_constant = np.expand_dims(gray_constant, 2)
        lab = np.concatenate([gray_constant, colors], axis=2)
        rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
        # it has shape [256, 4 * 256, 3]

        gray = Image.fromarray(gray).convert('RGB')
        rgb = Image.fromarray(rgb)

        image.paste(gray, (0, 256 * i))
        image.paste(rgb, (256 * 4, 256 * i))

    return image

In [ ]:
show_batch(y)